<a href="https://colab.research.google.com/github/Kshitij0412/ML-projects/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
    # import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-03-14 17:44:35--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  20.3MB/s    in 1.2s    

2024-03-14 17:44:37 (20.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books.head()


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.shape

(271379, 3)

In [ ]:
df_ratings.shape

(1149780, 3)

In [ ]:
df_books.isnull().sum()

isbn      0
title     0
author    1
dtype: int64

In [ ]:
df_ratings.isnull().sum()

user      0
isbn      0
rating    0
dtype: int64

In [ ]:
df_books.dropna(inplace=True)

In [ ]:
df_books.isnull().sum()

isbn      0
title     0
author    0
dtype: int64

In [ ]:
ratings = df_ratings['user'].value_counts()
ratings.sort_values(ascending=False).head()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
Name: user, dtype: int64

In [ ]:
len(ratings[ratings < 200])

104378

In [ ]:
df_ratings['user'].isin(ratings[ratings < 200].index).sum()

622224

In [ ]:
df_ratings_rm = df_ratings[
  ~df_ratings['user'].isin(ratings[ratings < 200].index)
]
df_ratings_rm.shape

(527556, 3)

In [ ]:
ratings  = df_ratings['isbn'].value_counts()
ratings.sort_values(ascending=False).head()

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
Name: isbn, dtype: int64

In [ ]:
len(ratings[ratings<100])

339825

In [ ]:
df_ratings_rm=df_ratings[
    ~df_ratings['isbn'].isin(ratings[ratings<100].index)
]
df_ratings_rm.shape

(138461, 3)

In [ ]:
df = df_ratings_rm.pivot_table(index=['user'],columns=['isbn'],values='rating').fillna(0).T
df.head()

user,9,14,16,26,39,42,44,51,67,75,...,278800,278807,278813,278819,278828,278832,278836,278843,278844,278854
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0


In [ ]:
books = ["Where the Heart Is (Oprah's Book Club (Paperback))",
        "I'll Be Seeing You",
        "The Weight of Water",
        "The Surgeon",
        "I Know This Much Is True"]

for book in books:
  print(df_ratings_rm.isbn.isin(df_books[df_books.title == book].isbn).sum())

585
140
117
133
242


In [ ]:
df.index = df.join(df_books.set_index('isbn'))['title']

In [ ]:
df = df.sort_index()
df.head()

user,9,14,16,26,39,42,44,51,67,75,...,278800,278807,278813,278819,278828,278832,278836,278843,278844,278854
title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.loc["The Queen of the Damned (Vampire Chronicles (Paperback))"][:5]

user
9     0.0
14    0.0
16    0.0
26    0.0
39    0.0
Name: The Queen of the Damned (Vampire Chronicles (Paperback)), dtype: float32

In [ ]:
model = NearestNeighbors(metric='cosine')
model.fit(df.values)

NearestNeighbors(metric='cosine')

In [ ]:
df.iloc[0].shape

(32712,)

In [ ]:
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
df.loc[title].shape

(32712,)

In [ ]:
distance, indice = model.kneighbors([df.loc[title].values], n_neighbors=6)

print(distance)
print(indice)

[[0.         0.56204545 0.59537727 0.71904826 0.84905005 0.875565  ]]
[[612 648 660 272 297 667]]


In [ ]:
df.iloc[indice[0]].index.values

array(['The Queen of the Damned (Vampire Chronicles (Paperback))',
       'The Tale of the Body Thief (Vampire Chronicles (Paperback))',
       'The Vampire Lestat (Vampire Chronicles, Book II)',
       'Interview with the Vampire',
       'Lasher: Lives of the Mayfair Witches (Lives of the Mayfair Witches)',
       'The Witching Hour (Lives of the Mayfair Witches)'], dtype=object)

In [ ]:
pd.DataFrame({
    'title'   : df.iloc[indice[0]].index.values,
    'distance': distance[0]
}) \
.sort_values(by='distance', ascending=False)

,title,distance
5,The Witching Hour (Lives of the Mayfair Witches),0.875565
4,Lasher: Lives of the Mayfair Witches (Lives of...,0.849050
3,Interview with the Vampire,0.719048
2,"The Vampire Lestat (Vampire Chronicles, Book II)",0.595377
1,The Tale of the Body Thief (Vampire Chronicles...,0.562045
0,The Queen of the Damned (Vampire Chronicles (P...,0.000000


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  try:
    book = df.loc[title]
  except KeyError as e:
    print('The given book', e, 'does not exist')
    return

  distance, indice = model.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : df.iloc[indice[0]].index.values,
      'distance': distance[0]
    }) \
    .sort_values(by='distance', ascending=False) \
    .head(5).values


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

[['The Witching Hour (Lives of the Mayfair Witches)' 0.8755649924278259]
 ['Lasher: Lives of the Mayfair Witches (Lives of the Mayfair Witches)'
  0.8490500450134277]
 ['Interview with the Vampire' 0.719048261642456]
 ['The Vampire Lestat (Vampire Chronicles, Book II)' 0.5953772664070129]
 ['The Tale of the Body Thief (Vampire Chronicles (Paperback))'
  0.5620454549789429]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()